# Phase 2, 3 and 4: Search, Augment and Generate the Answer
In this notebook there are several parts:
- Import libraries, load configuration variables and create clients
- Hybrid search with Semantic ranker
- Filter the chunks leaving the most relevant compared with the user's question
- Generate the answer for the query using the most relevante chunks as the context

### Import libraries, load configuration variables and create clients

In [ ]:
#%pip install python-dotenv
#%pip install openai
#%pip install tiktoken
#%pip install azure-search-documents

In [ ]:
# Import libraries
import os
import sys
import json
import time
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI

sys.path.append(os.path.abspath('..'))
from common_utils import *

# Load Azure OpenAI and AI Search variables and create clients
openai_config, ai_search_config = load_config()


aoai_endpoint: https://openai-asc-swit-north.openai.azure.com/
aoai_deployment_name: gpt-4o
oai_embedding_model: ada
aoai_rerank_model: gpt-4o-mini
ai_search_index_name_regs: rag-index-regs2
ai_search_index_name_docs: rag-index-docs2


### Search in AI Search with hybrid (keyword and vector searches) with semantic ranker

In [ ]:
# Generate the query for the question
question = "What is included in my Northwind Health Plus plan?"

# Hybrid search
results, num_results = semantic_hybrid_search(ai_search_config["ai_search_client_docs"],
                                              openai_config["openai_client"],
                                              openai_config["aoai_embedding_model"],
                                              question, 10)
print(f"num results: {num_results}")
print(f"num len(results): {len(results)}")
show_results(results, question)


#### Filter the chunks compared with the user's question and Generate the answer using the relevant chunks as context

In [ ]:
# Valid chunks for the user question
valid_chunks, num_chunks = get_filtered_chunks(openai_config["openai_client"],
                                               openai_config["aoai_rerank_model"],
                                               results,
                                               question)
print(f"num valid chunks: {num_chunks}")

# Generate answer:
answer = generate_answer(openai_config["openai_client"],
                         openai_config["aoai_deployment_name"],
                                       valid_chunks,
                                       question)
print(f"\n>> Answer:\n{answer}")


In [ ]:
## End-to-end process:

question = "What is included in my Northwind Health Plus plan?"
print(f'Question: {question}')

# Hybrid search with Semantic ranker
results, num_results = semantic_hybrid_search(ai_search_config["ai_search_client_docs"],
                                              openai_config["openai_client"],
                                              openai_config["aoai_embedding_model"],
                                              question, 50)
print(f"num results: {num_results}")
show_results(results, question)

# Filter valid chunks for the user question
valid_chunks, num_chunks = get_filtered_chunks(openai_config["openai_client"],
                                               openai_config["aoai_rerank_model"],
                                               results, question)

# Generate answer:
answer = generate_answer(openai_config["openai_client"],
                         openai_config["aoai_deployment_name"],
                         valid_chunks, question)
print(f"\n>> Answer: {answer}")